In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import cv2

annots = loadmat('devkit/cars_test_annos.mat')
class_names = [i.flat[0] for i in annots['annotations'][0]['fname']]
data = [[row.flat[0] for row in line] for line in annots['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'fname']
df_train = pd.DataFrame(data, columns=columns)

def convert_labels(path, x1, y1, x2, y2):
    """
    Definition: Parses label files to extract label and bounding box
        coordinates.  Converts (x1, y1, x1, y2) KITTI format to
        (x, y, width, height) normalized YOLO format.
    """
    def sorting(l1, l2):
        if l1 > l2:
            lmax, lmin = l1, l2
            return lmax, lmin
        else:
            lmax, lmin = l2, l1
            return lmax, lmin
        
    size = get_img_shape(path)
    xmax, xmin = sorting(x1, x2)
    ymax, ymin = sorting(y1, y2)
    dw = 1./size[1]
    dh = 1./size[0]
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def get_img_shape(path):
    path = 'test/'+path
    img = cv2.imread(path)
    try:
        return img.shape
    except AttributeError:
        print('error! ', path)
        return (None, None, None)

for index, row in df_train.iterrows():
    x1 = row[0]
    y1 = row[1]
    x2 = row[2]
    y2 = row[3]
    path = row[4]
    
    tuple_ = convert_labels(path, x1, y1, x2, y2)
    
    textfileLocation = 'test/' + path[:-4] + '.txt'
    file = open(textfileLocation, "w")
    any_string = str(tuple_[0]) + ' ' + str(tuple_[1]) + ' ' + str(tuple_[2]) + ' ' + str(tuple_[3])
    file.write(any_string)
    file.close()